In [2]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [4]:
# Import block
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
from tensorflow import keras
from keras.layers import Dense, Flatten
from keras.models import Sequential

In [5]:
# Dataset set-up
dataset_name = "imagenette320"
data_dir='/content/gdrive/MyDrive/imagenette320'

# Download the dataset
# dataset = tfds.load(dataset_name, data_dir, shuffle_files=True)

In [6]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory (
    data_dir,
    subset="training",
    validation_split=0.2,
    label_mode="categorical",
    seed=123,
    image_size=(320, 320),
    batch_size=32)

Found 13424 files belonging to 10 classes.
Using 10740 files for training.


In [7]:
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    subset="validation",
    validation_split=0.2,
    label_mode="categorical",
    seed=123,
    image_size=(320, 320),
    batch_size=32)

Found 13424 files belonging to 10 classes.
Using 2684 files for validation.


In [13]:
# Define the transfer learning model
resnet_model = Sequential()

pretrained_model = tf.keras.applications.ResNet50(
    include_top=False, #Changed from true
    input_shape=(320,320,3), #Image shape
    pooling='avg', #Global average pooling
    classes=10, #Number of classes of the dataset
    weights='imagenet', #Pre-trained weights
)
for layer in pretrained_model.layers:
    layer.trainable = False #Every layer inside is not gonna be trained, we are using the pre-trained weights

94765736/94765736 [==============================] - 1s 0us/step


In [14]:
resnet_model.add(pretrained_model) #Add the pre-trained model to the new model
resnet_model.add(Flatten()) #Flatten the output of the pre-trained model
resnet_model.add(Dense(512, activation='relu')) #Add a dense layer with 512 neurons and relu activation function <- fun can be changed to improve the model
resnet_model.add(Dense(10, activation='softmax')) #Add a dense layer with 10 neurons (output) and softmax activation function <- fun can be changed to improve the model

In [15]:
resnet_model.summary() #Print the model summary, now the model is ready to be compiled

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 2048)              23587712  
                                                                 
 flatten_1 (Flatten)         (None, 2048)              0         
                                                                 
 dense_2 (Dense)             (None, 512)               1049088   
                                                                 
 dense_3 (Dense)             (None, 10)                5130      
                                                                 
Total params: 24,641,930
Trainable params: 1,054,218
Non-trainable params: 23,587,712
_________________________________________________________________


In [17]:
resnet_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy']) #Compile the model with the optimizer, loss function and metrics (try others)

In [ ]:
history = resnet_model.fit(train_ds, epochs=10, validation_data=val_ds) #Train the model with the training dataset and validate it with the validation dataset

Epoch 1/10
336/336 [==============================] - 90s 251ms/step - loss: 0.1740 - accuracy: 0.9477 - val_loss: 0.1083 - val_accuracy: 0.9661
Epoch 2/10
336/336 [==============================] - 84s 249ms/step - loss: 0.0521 - accuracy: 0.9820 - val_loss: 0.0817 - val_accuracy: 0.9765
Epoch 3/10
336/336 [==============================] - 84s 249ms/step - loss: 0.0276 - accuracy: 0.9900 - val_loss: 0.1096 - val_accuracy: 0.9728
Epoch 4/10
130/336 [==========>...................] - ETA: 41s - loss: 0.0360 - accuracy: 0.9865

In [ ]:
fig = plt.gcf()
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.axis(ymin=0.5,ymax=1)
plt.grid()
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'])
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.grid()
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'])
plt.show()